In [164]:
from langgraph.graph import StateGraph, START,END
from typing import TypedDict, Annotated
from langchain_openai import ChatOpenAI
import operator
from dotenv import load_dotenv

In [165]:
load_dotenv()
chat_openai = ChatOpenAI(model="gpt-4o-mini", temperature=0.1, max_tokens=1000)


In [166]:
class Sentiment_Analyzer(TypedDict):
    # input_text: Annotated[list[str],operator.add]
    translate : Annotated[list[str],operator.add]
    sentiment : str
    Summarize : str
    combined_result : str

In [167]:
def translate_to_english(state: Sentiment_Analyzer) -> Sentiment_Analyzer:
    response = chat_openai.invoke(f"Translate to English:\n\n{state['translate']}")
    state['translate'] = response.content
    return state


In [168]:
def sentiment_analysis(state: Sentiment_Analyzer) -> Sentiment_Analyzer:
    response = chat_openai.invoke(f"Analyze the sentiment of the following text: {state['translate']}")
    state['sentiment'] = response.content
    return state

In [169]:
def summarize_text(state: Sentiment_Analyzer) -> Sentiment_Analyzer:
    response = chat_openai.invoke(f"Summarize the following text: {state['translate']}")
    state['Summarize'] = response.content
    return state

In [170]:
def combine_results(state: Sentiment_Analyzer) -> Sentiment_Analyzer:
    state['combined_result'] = f"Translation: {state['translate']}\nSentiment: {state['sentiment']}\nSummary: {state['Summarize']}"
    return state

In [171]:
workflow_graph = StateGraph(Sentiment_Analyzer)

workflow_graph.add_node("translate", translate_to_english)
workflow_graph.add_node("sentiment", sentiment_analysis)
workflow_graph.add_node("summarize", summarize_text)
workflow_graph.add_node("combine", combine_results)

# Workflow edges
workflow_graph.set_entry_point("translate")
workflow_graph.add_edge("translate", "sentiment")
workflow_graph.add_edge("translate", "summarize")
workflow_graph.add_edge("sentiment", "combine")
workflow_graph.add_edge("summarize", "combine")
workflow_graph.set_finish_point("combine")


In [172]:
workflow = workflow_graph.compile()
input_data = {"input_text": "Bonjour, comment ça va?"}
result = workflow.invoke(input_data)
print(result)



TypeError: can only concatenate list (not "str") to list